In [1]:
# Dependencies

import pandas as pd
import requests
import json
import pprint
import numpy as np
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
from us import states

# Census & gmaps API Keys
from config import (api_key, gkey)
year = 2012
c = Census(api_key, year)

# Configure gmaps
gmaps.configure(api_key=gkey)

In [2]:
datalst = []

In [3]:
for year in range(2012, 2018):
    c = Census(api_key, year) 

    data = c.acs5.get(('NAME', "B25077_001E","B25064_001E", "B15003_022E","B19013_001E"), 
                      {'for': 'zip code tabulation area:*'}) 

    cleandata = (pd.DataFrame(data) 
                   .rename(columns={"NAME": "Name", 
                                    "zip code tabulation area": "Zipcode", 
                                    "B25077_001E": "Median_Home_Value", 
                                    "B25064_001E": "Median_Rent", 
                                    "B15003_022E": "Bachelor_Degrees", 
                                    "B19013_001E": "Median_Income"}) 
                   .astype({'Zipcode':'int64'}) 
                   .query('(Median_Home_Value > 0) & (Median_Rent > 0) & (Median_Income > 0)')
                   .dropna()
                   .assign(Year = year)
                 )

    datalst.append(cleandata)


final_data = pd.concat(datalst, ignore_index = True)

In [4]:
final_data


,Name,Median_Home_Value,Median_Rent,Bachelor_Degrees,Median_Income,Zipcode,Year
0,ZCTA5 02322,310500.0,1131.0,456.0,70481.0,2322,2012
1,ZCTA5 02324,333900.0,1144.0,3856.0,90722.0,2324,2012
2,ZCTA5 02330,269100.0,1547.0,1178.0,67963.0,2330,2012
3,ZCTA5 02332,573900.0,957.0,3949.0,117197.0,2332,2012
4,ZCTA5 02333,325700.0,993.0,1430.0,80169.0,2333,2012
5,ZCTA5 02338,312800.0,1229.0,650.0,77905.0,2338,2012
6,ZCTA5 02339,459000.0,916.0,2874.0,99856.0,2339,2012
7,ZCTA5 02341,338300.0,1009.0,1149.0,92736.0,2341,2012
8,ZCTA5 02343,277600.0,1152.0,1359.0,61825.0,2343,2012
9,ZCTA5 02346,311300.0,1081.0,2775.0,74764.0,2346,2012
